In [75]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')


def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        if row.find("td") is not None:
            col = row.find_all('td')
            bank_name = col[1].find_all('a')[1]['title']
            market_cap = col[2].contents[0][:-1]
            data_dict = {"Name": bank_name,
                        "MC_USD_millions": float(market_cap)}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1],ignore_index=True)
    return df



def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']
    df["MC_GBP_millions"] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_millions']]
    df["MC_EUR_millions"] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_millions']]
    df["MC_INR_millions"] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_millions']] 
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_millions"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './exchange_rate.csv'
output_path = './Largest_banks_data.csv'

log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, csv_path)

log_progress("Data transformation complete. Initiating Loading process")

load_to_csv(df, output_path)

log_progress("Data saved to CSV file")

sql_connection = sqlite3.connect('Banks.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_millions) FROM {table_name}"
run_query(query_statement, sql_connection)


query_statement = f"SELECT Name from Largest_banks LIMIT 5"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

log_progress('Server Connection closed')



SELECT * from Largest_banks
                                      Name  MC_USD_millions  MC_GBP_millions  \
0                           JPMorgan Chase           432.92           346.34   
1                          Bank of America           231.52           185.22   
2  Industrial and Commercial Bank of China           194.56           155.65   
3               Agricultural Bank of China           160.68           128.54   
4                                HDFC Bank           157.91           126.33   
5                              Wells Fargo           155.87           124.70   
6                                     HSBC           148.90           119.12   
7                           Morgan Stanley           140.83           112.66   
8                  China Construction Bank           139.82           111.86   
9                            Bank of China           136.81           109.45   

   MC_EUR_millions  MC_INR_millions  
0           402.62         35910.71  
1           215

In [41]:

df["MC_GBP_millions"] = [np.round(x*0.8,2) for x in df['MC_USD_millions']]
df["MC_EUR_millions"] = [np.round(x*0.93,2) for x in df['MC_USD_millions']]
df["MC_INR_millions"] = [np.round(x*82.95,2) for x in df['MC_USD_millions']]

In [42]:
df

,Name,MC_USD_millions,MC_GBP_millions,MC_EUR_millions,MC_INR_millions
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [48]:
df

,Name,MC_USD_millions
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [60]:
help(sqlite3)


Help on package sqlite3:

NAME
    sqlite3

MODULE REFERENCE
    https://docs.python.org/3.11/library/sqlite3.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    The sqlite3 extension module provides a DB-API 2.0 (PEP 249) compliant
    interface to the SQLite library, and requires SQLite 3.7.15 or newer.
    
    To use the module, start by creating a database Connection object:
    
        import sqlite3
        cx = sqlite3.connect("test.db")  # test.db will be created or opened
    
    The special path name ":memory:" can be provided to connect to a transient
    in-memory database:
    
        cx = sqlite3.connect(":memory:")  # connect to a database in RAM
    
    Once a connection has 